# Credentials and initialize conneciton

Follow https://developers.google.com/earth-engine/reference/Quickstart#before-you-begin to get a secret key

In [2]:
import json

KEY = 'my-secret-key.json'
with open(KEY, 'r') as f:
    data = json.load(f)
    SERVICE_ACCOUNT = data['client_email']
    PROJECT = data['project_id']

In [3]:
import ee

ee_creds = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
ee.Initialize(ee_creds)

# NDVI around coordinate

In [6]:
import geemap

def add_ndvi_and_index_before_mask(image, index):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    index_band = ee.Image.constant(index).rename('index').toFloat()
    return image.addBands([ndvi, index_band])

def mask_s2_bad_data(image):
    scl = image.select('SCL')
    bad_classes = [0, 1, 2, 3, 8, 9, 10]
    mask = scl.remap(bad_classes, [0]*len(bad_classes), 1).eq(1)
    return image.updateMask(mask)

# --- 2. Define geometry and load collection ---

geom = ee.Geometry.BBox(38.4, -2.9, 38.7, -2.6)

collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
    .filterBounds(geom)
    .filterDate('2024-05-10', '2024-06-10')
    .sort('system:time_start', True)  # Oldest first
)

images_list = collection.toList(collection.size())
dates = []
processed_images = []

# --- 3. Build image list with NDVI + index BEFORE masking ---
for i in range(images_list.size().getInfo()):
    img = ee.Image(images_list.get(i))
    date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    dates.append(date_str)

    img_with_bands = add_ndvi_and_index_before_mask(img, i)
    img_masked = mask_s2_bad_data(img_with_bands)

    processed_images.append(img_masked)

indexed_collection = ee.ImageCollection(processed_images)

# --- 4. Build final filled composite ---
filled = indexed_collection.qualityMosaic('index').clip(geom)
ndvi = filled.select('NDVI')
index = filled.select('index')

# Also get the newest unmasked image for comparison
newest_raw = collection.sort('system:time_start', False).first().clip(geom)
rgb_raw = newest_raw.select(['B4', 'B3', 'B2']).divide(10000)
rgb_filled = filled.select(['B4', 'B3', 'B2']).divide(10000)

# --- 5. Visualization parameters ---
ndvi_vis = {
    'min': 0,
    'max': 1.0,
    'palette': ['red', 'yellow', 'green']
}

index_vis = {
    'min': 0,
    'max': len(dates) - 1,
    'palette': ['black', 'white']
}

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
}

# --- 6. Create map ---
Map = geemap.Map(center=[-2.75, 38.55], zoom=12)
Map.addLayer(ndvi, ndvi_vis, 'Cloud-Filled NDVI')
Map.addLayer(index, index_vis, 'Image Recency Index (black=newest)')
Map.addLayer(rgb_raw, rgb_vis, 'RGB Composite (Raw)')
Map.addLayer(rgb_filled, rgb_vis, 'RGB Composite (Filtered)')

# --- 7. Print date mapping ---
print("Index to Date mapping:")
for idx, d in enumerate(dates):
    print(f"Index {idx}: {d}")

Map


Index to Date mapping:
Index 0: 2024-05-14
Index 1: 2024-05-14
Index 2: 2024-05-19
Index 3: 2024-05-19
Index 4: 2024-05-24
Index 5: 2024-05-24
Index 6: 2024-05-29
Index 7: 2024-05-29
Index 8: 2024-06-03
Index 9: 2024-06-08
Index 10: 2024-06-08


Map(center=[-2.75, 38.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

# Dynamic World around coordinate

In [7]:
import geemap
from pprint import pprint

# Create map
Map = geemap.Map(center=[-2.75, 38.55], zoom=12)

# Define geometry and date range
geom = ee.Geometry.BBox(38.4, -2.9, 38.7, -2.6)

# Filter Dynamic World collection
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(geom) \
    .filterDate('2024-05-10', '2024-06-10')

dw_img = dw_col.qualityMosaic('label').clip(geom)

# Define color palette for Dynamic World labels
vis_palette = [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635',
    'dfc35a', 'c4281b', 'a59b8f', 'b39fe1'
]

# Visualize the label band
dw_vis = dw_img.select('label').visualize(min=0, max=8, palette=vis_palette)

# Add layer to the map
Map.addLayer(ndvi, ndvi_vis, 'Cloud-Filled NDVI')
Map.addLayer(index, index_vis, 'Image Recency Index (black=newest)')
Map.addLayer(rgb_raw, rgb_vis, 'RGB Composite (Raw)')
Map.addLayer(rgb_filled, rgb_vis, 'RGB Composite (Filtered)')
Map.addLayer(dw_vis, {}, 'Dynamic World label')
Map


Map(center=[-2.75, 38.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…